In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv('villamhub_rag_dataset.csv')
df.head(3)

,id,text,source
0,1,Villam Hub is an urban farming platform focuse...,VillamHub_Overview
1,2,The founder of Villam Hub is Wisdom Chibuzor. ...,VillamHub_Overview
2,3,Villam Hub was created by a team of young inno...,VillamHub_Overview


In [3]:
from sentence_transformers import SentenceTransformer

#  Load embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')  # gives 384-dimensional embeddings

In [4]:
from pinecone import Pinecone

# Connect to Pinecone using API key 
pc = Pinecone(api_key="pcsk_6BJeTV_8F1pjuyoyrb8FDK2QqrPKaosRuuHJrpUx52WwX44q7z6JmWqMuDb7R8SfJtSpzu")

# Connect to  existing index
index = pc.Index("villam-chatbot")


In [5]:
# Convert text to embeddings and prepare for Pinecone

vectors = []
for i, row in df.iterrows():   # goes through each row in the dataset 
    # convert text to embedding
    embedding = model.encode(row['text']).tolist()  
    
    # Create a Pinecone-friendly vector entry
    vectors.append({
        'id': f'id-{i}',
        'values': embedding,
        'metadata': {'text': row['text'], 'source': row['source']}
    })


# send data to pinecone
index.upsert(vectors=vectors, namespace="default")

C:\Users\Olamide\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'upserted_count': 20}

In [6]:

# Sample user query
query = "What is Villam Hub and what does it do?"

# Embed the query
query_embedding = model.encode(query).tolist()

# Search Pinecone for similar entries
response = index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True,
    namespace="default"   # ← this is important
)

# Show the results
for match in response['matches']:
    print(f"Score: {match['score']}")
    print(f"Text: {match['metadata']['text']}\n")


Score: 0.843910694
Text: Villam Hub is an urban farming platform focused on helping people grow food, plant trees, and live more sustainably. It combines digital technology with offline experiences.

Score: 0.70687741
Text: Villam Hub was created by a team of young innovators from Altschool Africa, committed to environmental and food system transformation.

Score: 0.66407156
Text: The founder of Villam Hub is Wisdom Chibuzor. He is a Product Manager and a farmer who is passionate about sustainable agriculture.

Score: 0.658494174
Text: Villam Hub supports sustainable agriculture through education, tools, and community engagement, empowering users to take part in eco-friendly food systems.

Score: 0.631631
Text: Villam Hub offers affordable farming kits, including hydroponic kits, irrigation kits, and backyard garden kits, to promote urban agriculture.



In [7]:
query = "Who is Villam Hub founder?"

# Embed the query
query_embedding = model.encode(query).tolist()

# Search Pinecone for similar entries
response = index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True,
    namespace="default"   # ← this is important
)

# Show the results
for match in response['matches']:
    print(f"Score: {match['score']}")
    print(f"Text: {match['metadata']['text']}\n")

Score: 0.828174889
Text: The founder of Villam Hub is Wisdom Chibuzor. He is a Product Manager and a farmer who is passionate about sustainable agriculture.

Score: 0.68276912
Text: Villam Hub was created by a team of young innovators from Altschool Africa, committed to environmental and food system transformation.

Score: 0.668948889
Text: Villam Hub is an urban farming platform focused on helping people grow food, plant trees, and live more sustainably. It combines digital technology with offline experiences.

Score: 0.538108468
Text: Villam Hub’s mission is to build an environmentally aware generation and create a system that promotes accessible and sustainable food growing.

Score: 0.530345321
Text: Villam Hub supports sustainable agriculture through education, tools, and community engagement, empowering users to take part in eco-friendly food systems.

